# Object Detect - Image demo using tflite

In [1]:
import tflite_runtime.interpreter as tflite

In this notebook, we will use model tflite object detect

### Load Trained Model

coco detect model  ``detect.tflite``
> Please make sure the file has uploaded fully before calling the next cell

In [2]:
import numpy as np
import cv2
from pathlib import Path

In [3]:
interpreter = tflite.Interpreter(model_path='tflite_model/detect.tflite')

> Load detect label file

In [4]:
with open('tflite_model/labelmap.txt', 'r') as F:
    class_names = F.readlines()

### Pre-Processing Interpreter and test Image

tflite interpreter 

In [5]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

prepare image data

In [6]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

# prepara input image
img = camera.value #cv2.imread('chairs.jpg')
x = cv2.resize(img, (300, 300))
x = x[:, :, [2,1,0]]  # BGR -> RGB
x = np.expand_dims(x, axis=0) # 3->4

set input tensor

In [10]:
import time

start_time = time.monotonic()
interpreter.set_tensor(input_details[0]['index'], x)
interpreter.invoke()
elapsed_ms = (time.monotonic() - start_time) * 1000
print('detect time={}'.format(elapsed_ms))

detect time=365.1827260000573


get outpu tensor

In [8]:
tflite_results1 = interpreter.get_tensor(output_details[0]['index'])  
tflite_results2 = interpreter.get_tensor(output_details[1]['index'])  
tflite_results3 = interpreter.get_tensor(output_details[2]['index']) 
tflite_results4 = interpreter.get_tensor(output_details[3]['index'])  

### detect result

In [9]:
dstimg = cv2.resize(img, (480, 480))
for i in range(int(tflite_results4[0])):
    (top, left, bottom, right) = tflite_results1[0, i] * 300
    class_name = class_names[tflite_results2[0, i].astype(int)].rstrip()
    prob = tflite_results3[0, i]
    if prob >= 0.5:
        print("Location=({},{})-({},{})".format(int(left), int(top), int(right), int(bottom)))
        print("Class={}".format(class_name))
        print("Probability={}".format(prob))
        left = int(left * 480 / 300)
        right = int(right * 480 / 300)
        top =  int(top * 480 / 300)
        bottom = int(bottom * 480 / 300)
        cv2.rectangle(dstimg, (left, top), (right, bottom), (0, 0, 255), 1)

Location=(57,21)-(264,279)
Class=chair
Probability=0.80859375
Location=(13,54)-(121,184)
Class=chair
Probability=0.62109375
Location=(167,37)-(287,177)
Class=chair
Probability=0.59765625
Location=(51,25)-(169,175)
Class=chair
Probability=0.5859375
Location=(151,38)-(284,269)
Class=chair
Probability=0.55078125
Location=(92,93)-(263,284)
Class=chair
Probability=0.5


Show a result image 

In [ ]:
cv2.imshow("test image", dstimg)
cv2.moveWindow("test image", 0, 0)
cv2.waitKey(0)
cv2.destroyAllWindows()

End!